In [37]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver    
import time
import pandas as pd
import re
import collections
import nltk
from nltk import word_tokenize, Text, FreqDist
from nltk.corpus import wordnet as wn
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
wnl = nltk.WordNetLemmatizer()
import pathlib
from newspaper import Article
from urllib.parse import urlparse
# importing itertools for accumulate()
import itertools
# importing functools for reduce()
import functools
from tqdm.notebook import tqdm_notebook
import json
import praw
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import os
import concurrent.futures
import time
from tqdm import tqdm
import gc
from nltk.corpus import stopwords
from IPython.display import display, clear_output
stop_words = set(stopwords.words('english'))
import pickle

# Selenium Web Crawler

In [38]:
'''

Input : Xpath for the dropdown list 
Output: Xpath for the choices in the dropdown

'''
def getNewsContentByArticle(Article, url):
    # get content 
    try:
        content = Article(url)
        content.download()
        content.parse()
        content = content.text
    except Exception as e:
        print(e)
        content = ""
    return content

def getNewsContentByGoogle(title):
    
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")

    # to search
    query = "news:" + title
 
    content, domain, url = "", "", ""
    
    count = 0
    
    try:
    
        for j in search(query):
            url = j

            domain = urlparse(j).netloc

            isIn = [True for c in channels if c in domain]

            content = getNewsContentByArticle(Article,j)

            count = count + 1

            if not isIn == [] or count > 10:
                break
                
    except:
        pass
    
    return {'content':content, 'domain':domain,'url':url}
        

def getTableItems(driver, xpath_items):
    try:
        items = driver.find_elements("xpath", xpath_items)
    except Exception as e:
        items = []
        print(e)
        pass
    return items

def getDropdownChoices(driver, xpath_dropdown, xpath_choices):
    # Wait for initialize, in seconds
    wait = WebDriverWait(driver, 10)

    dropdown = wait.until(EC.visibility_of_element_located((By.XPATH, xpath_dropdown)))
    
    time.sleep(1)
    
    dropdown.click()
    
    #2. find the choices on the list
    try:
        dropdown = driver.find_elements("xpath",xpath_choices)
    except ElementClickInterceptedException:
        print('\n-- No dropdown list found --')
        pass
    
    return dropdown

def getChildElement(node, xpath):
    
    xpath = ".//descendant-or-self::" + xpath
    
    try:
        child_node = node.find_element("xpath", xpath)
    except NoSuchElementException:
        print("\n-- Unable to find the child element")
        raise
        
    return child_node


def clickToGo(driver, xpath):
    try:
        button = driver.find_element("xpath",xpath)
        button.click()
        time.sleep(1)
    except:
        #print('\n-- No element {} found --'.format(xpath))
        raise

In [39]:
def selenium_init():
    #initalise crawler option(s)
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    options.add_argument('headless')
    ROOT_DIR = pathlib.Path().absolute()
    print(ROOT_DIR)
    driver = webdriver.Chrome(executable_path=(str(ROOT_DIR)+"/chromedriver"), options=options)
    driver.set_window_size(1120, 1000)
    
    return driver

## Getting Posts

In [4]:
def getRedditPostItems(driver, url,label):
    out= []

    driver.get(url)
    time.sleep(1)
    
    #1. click to the time series
    xpath_dropdown = "//div[contains(@class,'dropdown lightdrop')]"
    xpath_choices = "//DIV[contains(@class,'drop-choices')]//A[contains(@class,'choice')]"
    #1. allocate the dropdown menu and click the timeseries
    timeseries = getDropdownChoices(driver,xpath_dropdown, xpath_choices)
    timeseries = [choice for choice in timeseries if choice.text == label]
    
    if not timeseries == []:
        timeseries[0].click()
        time.sleep(2)
    else:
        pass
    
    #3. loop and retrieve items on the page.
    searching = True
    noOfDocs = 0
    while searching:
        #2. load main page and pick relative items
        xpath_main_page_items = "//div[contains(@class,'thing id-t3')  and .//SPAN/@title='News']"
        
        items = getTableItems(driver, xpath_main_page_items)
    
        for item in items:
            # get the post id
            try:
                post_id = item.get_property("id")
            except NoSuchElementException:
                print('-- post id not found')
                post_id = ""
            # get the news title
            try:
                title = item.find_element("xpath",".//descendant-or-self::A[contains(@class,'title may-blank')]").text
            except NoSuchElementException:
                title = ""
            # get the news URL
            try:
                title_url = item.find_element("xpath",".//descendant-or-self::A[contains(@class,'title may-blank')]").get_property('href')
            except NoSuchElementException:
                title_url = ""
            # get the datetime
            try:
                datetime  = item.find_element("xpath", ".//descendant-or-self::p[contains(@class, 'tagline')]/time").get_attribute('datetime')
            except NoSuchElementException:
                datetime = ""
            # get the domain name
            try:
                domain = item.find_element("xpath", ".//descendant-or-self::SPAN[contains(@class, 'domain')]").text
            except NoSuchElementException:
                domain = ""

            domain = urlparse(title_url).netloc
            # get the news score
            try:
                scores = item.find_element("xpath", ".//descendant-or-self::div[contains(@class, 'score unvoted')]").text
            except NoSuchElementException:
                scores = ""
            # get the no of comments
            try:
                no_of_cmts = item.find_element("xpath", ".//descendant-or-self::li[contains(@class,'first')]/a").text.split()[0]
            except NoSuchElementException:
                no_of_cmts = "0"
            # get the comment URL
            try:
                cmt_url = item.find_element("xpath", ".//descendant-or-self::li[contains(@class,'first')]/a").get_property('href')
            except NoSuchElementException:
                cmt_url = ""
            # get the article content
            content = getNewsContentByArticle(Article, title_url)

            if content == "" or "reddit" in title_url:
                out_dict = getNewsContentByGoogle(title)
                content = out_dict['content']
                domain = out_dict['domain']
                title_url = out_dict['url'] #bug fixed for missing url
            
            # expand all clickable comment section
            
            one_instance = {
                'post_id': post_id,
                'post_title': title,
                'post_url':title_url,
                'post_domain':domain,
                'post_datetime': datetime,
                'post_score': scores,
                'post_no_of_cmts': no_of_cmts,
                'post_article': content,
                'cmt_url': cmt_url
            }
            
            out.append(one_instance)
            
            noOfDocs = noOfDocs + 1
            print('\n-- {} no of records have collected.'.format(noOfDocs))
            
        
        # click next to go
        xpath_next = "//div[contains(@class, 'nav-buttons')]//a[contains(text(), 'next')]"
        
        # Wait for initialize, in seconds
        try:
            clickToGo(driver, xpath_next)
        except:
            print('\n-- DEBUG: The web scraping is completed.')
            break
        

    return out


## Getting Comments for Each Post

In [10]:
def getCommentsList(items,p_id=""):
    
    out = []
    
    for idx, comment_item in enumerate(items):
        #1. comment id 
        try:
            c_id = (comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@class, 'comment')]").get_attribute("id")).split('_')[-1]
        except NoSuchElementException:
            c_id = ""
            
        #2. comment user
        try:
            c_user = comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@class,'entry unvoted')]//p[contains(@class, 'tagline')]/a[contains(@class,'author')]").text
        except NoSuchElementException:
            c_user = ''
            
        #3. comment datetime
        try:
            c_datetime = comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@class,'entry unvoted')]//p[contains(@class, 'tagline')]/time").get_attribute('datetime')
        except NoSuchElementException:
            c_datetime = ''
        
        #4. comment text
        try:
            c_text = comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@class, 'md')]").text
        except NoSuchElementException:
            c_text = ''
            
        #5. post score
        try:
            c_score = comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@class,'entry unvoted')]//p[contains(@class, 'tagline')]//span[contains(@class,'score unvoted')]").text
        except NoSuchElementException:
            c_score = ''
        
        one_comment_item ={
                'cmt_id' : c_id,
                'replyTo' : p_id,
                'cmt_user': c_user,
                'cmt_score':c_score,
                'cmt_text': c_text,
                'cmt_datetime': c_datetime
        }
        if c_id != '' and c_user != '' and c_text != '' and c_datetime != '':
            out.append(one_comment_item)
        #6. check if has child element:
        try:
            # This comment has a child element, recursion goes on:
            child_element = comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@class,'child')]")
            this_child = child_element.find_elements("xpath","./div[contains(@class,'sitetable listing')]/div[contains(@class,'thing') and not(contains(@class,'morechildren'))]")
            out.extend(getCommentsList(this_child, p_id=c_id))
        except Exception as e:
            # This comment does not has a child element:
            print(e)
            pass
    
    return out

def getRedditCommentItems(driver, c_url):
    driver.get(c_url)
    
    time.sleep(2.5)
    try:
        expanders = driver.find_elements("xpath", "//p[contains(@class, 'tag')]/a[text()='[+]']")
        for each in expanders:
            each.click()
    except NoSuchElementException:
        pass
    
    try:
        expanders = driver.find_elements("xpath", "//span[contains(text(),'more comments')]/a[text()='more comments']")
        for each in expanders:
            each.click()
    except NoSuchElementException:
        pass    
    
    xpath_comments_dir = "//div[contains(@class, 'sitetable nestedlisting')]/div[contains(@class, 'thing')]"
    
    #root_items = driver.find_elements("xpath", xpath_comments_dir)
    
    root_items = getTableItems(driver, xpath_comments_dir)
    
    out = getCommentsList(root_items, p_id="")
    return out

# Get New Data

url= "https://old.reddit.com/r/singapore/top/"
driver = selenium_init()
try:
    items = getRedditPostItems(driver, url,"past week")
except:
    print('\n -- DEBUG: Errors occured when collecting news/posts on reddit.')
    raise

for item in items:
    try:
        item['comments'] = getRedditCommentItems(driver, item['c_url'])
        time.sleep(1)
    except Exception as e:
        print('\n', item['c_url'])

with open('data/reddit-12.json', 'w', encoding='utf-8') as output_file:
    json.dump(items , output_file ,indent = 2)

# Load Data

In [11]:
from os import listdir
from os.path import isfile, join

def list_directory(path):
    onlyfiles = [path+f for f in listdir(path) if f.split('.')[-1] == 'json'] 
    print('\nINFO -- File names:',onlyfiles)
    return onlyfiles

def load_reddit_data(paths):
    dic = {}
    for path in paths:
        try:
            new = json.load(open(path,encoding="utf8"))
        except Exception as e:
            print('\n-- INFO : ',e, path)
        
        try:
            dic['data'].extend(new['data'])
        except Exception as e:
            print('\n-- INFO : ',e, path)
            dic = new
    return dic

In [12]:
reddit_data = load_reddit_data(list_directory('data/reddit/'))

reddit_df = pd.DataFrame.from_dict(reddit_data['data'],orient='columns')


INFO -- File names: ['data/reddit/reddit-02.json', 'data/reddit/reddit-03.json', 'data/reddit/reddit-04.json', 'data/reddit/reddit-05.json', 'data/reddit/reddit-06.json', 'data/reddit/reddit-07.json', 'data/reddit/reddit-08.json', 'data/reddit/reddit-09.json', 'data/reddit/reddit-10.json', 'data/reddit/reddit-11.json', 'data/reddit/reddit-12.json']

-- INFO :  'data' data/reddit/reddit-02.json


In [13]:
from tqdm.notebook import trange, tqdm
import concurrent.futures
import gc

url= "https://old.reddit.com/r/singapore/top/"

driver = selenium_init()

new_comments = {}

#driver.get("https://old.reddit.com/r/singapore/comments/sp1ex7/privileges_committee_recommends_s35000_fine_for/")

for idx, row in tqdm(reddit_df.iterrows(), total=reddit_df.shape[0]):
    c_url = row['c_url']
    try:
        new_item_cmts = getRedditCommentItems(driver, str(c_url))
    except Exception as e:
        print('\n-- INFO:', e, new_item_cmts)
        new_item_cmts = []
    
    new_comments[idx] = new_item_cmts

C:\Users\liux5\Documents\Project 3 - DSTA


C:\Users\liux5\AppData\Local\Temp\ipykernel_20788\1316614122.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=(str(ROOT_DIR)+"/chromedriver"), options=options)


  0%|          | 0/4854 [00:00<?, ?it/s]

In [14]:
with open("reddit_comments.json", "w") as outfile:
    json.dump(new_comments, outfile)

In [32]:
all_comments_df = {}

for idx, comments in new_comments.items():
    comments_df = pd.DataFrame.from_records(comments)
    all_comments_df[idx] = comments_df

In [40]:
def save_pickle_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
save_pickle_object(reddit_df,'news_crawler/output/df/new/REDDIT_EN_1-12.pikle')

In [41]:
save_pickle_object(all_comments_df,'comments_df')

In [46]:
save_pickle_object(reddit_df,'news_crawler/output/df/new/REDDIT_EN_1-12.pikle')

In [50]:
all_comments_df[0]

,cmt_id,replyTo,cmt_user,cmt_score,cmt_text,cmt_datetime
0,hwcguvl,,3ply,527 points,"""The committee did not recommend any action ag...",2022-02-10T09:02:43+00:00
1,hwci3ta,hwcguvl,ShadeX8,167 points,I’m running out of popcorn. How many more twis...,2022-02-10T09:19:30+00:00
2,hwcsrjg,hwci3ta,ShadeX8,102 points,In before it was all a plot from Jamus to take...,2022-02-10T11:39:10+00:00
3,hwcxoxq,hwcsrjg,twst222,63 points,He has his own Umbrella Academy,2022-02-10T12:32:10+00:00
4,hwdc1s8,hwcxoxq,jermso,16 points,ella,2022-02-10T14:30:01+00:00
...,...,...,...,...,...,...
186,hwcdvxp,,dittotan,29 points,PAP is really piss-poor at reading the room wh...,2022-02-10T08:23:10+00:00
187,hwce0ry,hwcdvxp,fateoftheg0dz,81 points,"Curious, what does the ""general public"" actual...",2022-02-10T08:24:52+00:00
188,hwch6eh,hwce0ry,Reclusivechair,63 points,"Good qn. I suspect that the ""general public"" m...",2022-02-10T09:07:04+00:00
189,hwcuppr,hwcdvxp,eddalton,9 points,Ironically this comment itself is piss-poor at...,2022-02-10T12:01:13+00:00
